# Learning Analytics (Ben)

This notebook focuses on user **`ben`** and the metrics we discussed:

- Words studied (unique, cumulative over time)
- Words learned (`retrievability >= R_TARGET`, cumulative-ever over time)
- Verb conjugation learned (point-in-time over time; remembered only when both perfectum and past tense are `>= R_TARGET`)
- Study time from **session span** (`max(timestamp) - min(timestamp)` per session, summed per day)

All dates are bucketed in **UTC**.


In [ ]:
import sys
from pathlib import Path

import numpy as np
import pandas as pd
from dotenv import load_dotenv

try:
    import plotly.graph_objects as go
    from plotly.subplots import make_subplots
except ModuleNotFoundError:
    get_ipython().run_line_magic('pip', 'install -q plotly')
    import plotly.graph_objects as go
    from plotly.subplots import make_subplots

from IPython.display import display

ROOT = Path.cwd()
if not (ROOT / 'core').exists():
    ROOT = ROOT.parent
load_dotenv(ROOT / '.env')
if str(ROOT) not in sys.path:
    sys.path.insert(0, str(ROOT))

from core import fsrs
from core.fsrs.constants import R_TARGET
from core.fsrs.database import get_session
from core.fsrs.models import ReviewEvent

USER_ID = 'ben'
WORD_EX = 'word_translation'
VERB_EX = ['verb_perfectum', 'verb_past_tense']
pd.set_option('display.max_rows', 200)


In [ ]:
session = get_session()
try:
    events = pd.read_sql(
        session.query(ReviewEvent).filter(ReviewEvent.user_id == USER_ID).statement,
        session.bind,
    )
finally:
    session.close()

if events.empty:
    raise ValueError(f'No review events found for user_id={USER_ID!r}.')

events['timestamp'] = pd.to_datetime(events['timestamp'], utc=True)
events = events.sort_values('timestamp').reset_index(drop=True)
events['event_day_utc'] = events['timestamp'].dt.floor('D')

start_day = events['event_day_utc'].min()
end_day = pd.Timestamp.now(tz='UTC').floor('D')
all_days = pd.date_range(start_day, end_day, freq='D', tz='UTC')

print(f'Loaded {len(events):,} events for user {USER_ID!r}')
print(f'Date range (UTC): {start_day.date()} -> {end_day.date()}')
display(events.head())


In [ ]:
def cumulative_from_first_dates(first_dates: pd.Series, day_index: pd.DatetimeIndex) -> pd.Series:
    if first_dates.empty:
        return pd.Series(0, index=day_index, dtype='int64')
    counts = first_dates.value_counts().sort_index()
    return counts.reindex(day_index, fill_value=0).cumsum().astype('int64')

def daily_session_span_hours(events_df: pd.DataFrame, exercise_types: list[str], day_index: pd.DatetimeIndex) -> pd.Series:
    scoped = events_df[
        events_df['exercise_type'].isin(exercise_types)
        & events_df['session_id'].notna()
    ].copy()
    if scoped.empty:
        return pd.Series(0.0, index=day_index, dtype='float64')

    spans = scoped.groupby('session_id').agg(
        session_start=('timestamp', 'min'),
        session_end=('timestamp', 'max'),
    )
    spans['span_hours'] = (spans['session_end'] - spans['session_start']).dt.total_seconds() / 3600.0
    spans['day_utc'] = spans['session_start'].dt.floor('D')

    daily = spans.groupby('day_utc')['span_hours'].sum()
    return daily.reindex(day_index, fill_value=0.0)

def daily_card_retrievability(events_df: pd.DataFrame, exercise_type: str, day_index: pd.DatetimeIndex) -> pd.DataFrame:
    scoped = events_df.loc[
        events_df['exercise_type'] == exercise_type,
        ['word_id', 'timestamp', 'is_ltm_event', 'stability_after']
    ].copy()

    if scoped.empty:
        return pd.DataFrame(columns=['day', 'word_id', 'retrievability'])

    scoped = scoped.sort_values(['word_id', 'timestamp']).reset_index(drop=True)

    snapshots = pd.DataFrame({'day': day_index})
    snapshots['snapshot_ts'] = snapshots['day'] + pd.Timedelta(days=1) - pd.Timedelta(microseconds=1)

    out_parts = []

    for word_id, grp in scoped.groupby('word_id', sort=False):
        g = grp.sort_values('timestamp').copy()
        g['last_ltm_ts'] = g['timestamp'].where(g['is_ltm_event'].astype(bool)).ffill()

        merged = pd.merge_asof(
            snapshots.sort_values('snapshot_ts'),
            g[['timestamp', 'stability_after', 'last_ltm_ts']].sort_values('timestamp'),
            left_on='snapshot_ts',
            right_on='timestamp',
            direction='backward',
        )

        valid = merged['stability_after'].notna() & merged['last_ltm_ts'].notna()
        if not valid.any():
            continue

        days_since = (merged.loc[valid, 'snapshot_ts'] - merged.loc[valid, 'last_ltm_ts']).dt.total_seconds() / 86400.0
        retr = np.exp(-days_since / merged.loc[valid, 'stability_after'])

        out_parts.append(pd.DataFrame({
            'day': merged.loc[valid, 'day'].to_numpy(),
            'word_id': word_id,
            'retrievability': retr.to_numpy(),
        }))

    if not out_parts:
        return pd.DataFrame(columns=['day', 'word_id', 'retrievability'])

    return pd.concat(out_parts, ignore_index=True)


In [ ]:
# ---- Words track ----
word_events = events[events['exercise_type'] == WORD_EX].copy()

word_first_studied = word_events.groupby('word_id')['timestamp'].min().dt.floor('D')
word_studied_cum = cumulative_from_first_dates(word_first_studied, all_days)

word_state_events = word_events[['word_id', 'timestamp', 'is_ltm_event', 'stability_after']].copy()
word_state_events = word_state_events.sort_values(['word_id', 'timestamp'])
word_state_events['last_ltm_ts'] = word_state_events['timestamp'].where(word_state_events['is_ltm_event'].astype(bool))
word_state_events['last_ltm_ts'] = word_state_events.groupby('word_id')['last_ltm_ts'].ffill()
word_state_events['days_since_ltm'] = (word_state_events['timestamp'] - word_state_events['last_ltm_ts']).dt.total_seconds() / 86400.0
word_state_events['retr_after'] = np.exp(-word_state_events['days_since_ltm'] / word_state_events['stability_after'])

word_first_learned = (
    word_state_events[word_state_events['retr_after'] >= R_TARGET]
    .groupby('word_id')['timestamp']
    .min()
    .dt.floor('D')
)
word_learned_cum = cumulative_from_first_dates(word_first_learned, all_days)

word_snaps = fsrs.get_all_cards_with_state(WORD_EX, USER_ID)
word_snap_df = pd.DataFrame([{'word_id': s.word_id, 'retrievability': s.retrievability} for s in word_snaps])

current_words_studied = int(word_events['word_id'].nunique())
current_words_learned = int((word_snap_df['retrievability'] >= R_TARGET).sum()) if not word_snap_df.empty else 0

word_span_daily = daily_session_span_hours(events, [WORD_EX], all_days)
word_span_cum = word_span_daily.cumsum()

print('Words track KPIs')
print('----------------')
print(f'Current words studied (unique): {current_words_studied:,}')
print(f'Current words learned (retrievability >= {R_TARGET:.2f}): {current_words_learned:,}')


In [ ]:
# ---- Conjugation track ----
perf_daily = daily_card_retrievability(events, 'verb_perfectum', all_days)
past_daily = daily_card_retrievability(events, 'verb_past_tense', all_days)

conj_daily = perf_daily.merge(
    past_daily,
    on=['day', 'word_id'],
    suffixes=('_perf', '_past'),
    how='inner',
)

if conj_daily.empty:
    conjugation_learned_point_in_time = pd.Series(0, index=all_days, dtype='int64')
else:
    conj_daily['remembered'] = (
        (conj_daily['retrievability_perf'] >= R_TARGET)
        & (conj_daily['retrievability_past'] >= R_TARGET)
    )
    conjugation_learned_point_in_time = (
        conj_daily[conj_daily['remembered']]
        .groupby('day')['word_id']
        .nunique()
        .reindex(all_days, fill_value=0)
        .astype('int64')
    )

perf_snap = pd.DataFrame([{'word_id': s.word_id, 'r_perf': s.retrievability} for s in fsrs.get_all_cards_with_state('verb_perfectum', USER_ID)])
past_snap = pd.DataFrame([{'word_id': s.word_id, 'r_past': s.retrievability} for s in fsrs.get_all_cards_with_state('verb_past_tense', USER_ID)])

conj_current = perf_snap.merge(past_snap, on='word_id', how='outer').fillna(0.0)
current_conj_learned = int(((conj_current['r_perf'] >= R_TARGET) & (conj_current['r_past'] >= R_TARGET)).sum()) if not conj_current.empty else 0

conj_span_daily = daily_session_span_hours(events, VERB_EX, all_days)
conj_span_cum = conj_span_daily.cumsum()

print('Conjugation track KPIs')
print('----------------------')
print(f'Current conjugation learned (both tenses >= {R_TARGET:.2f}): {current_conj_learned:,}')


In [ ]:
# Toggle here: 'words' or 'conjugation'
TRACK = 'words'

if TRACK not in {'words', 'conjugation'}:
    raise ValueError("TRACK must be 'words' or 'conjugation'")

if TRACK == 'words':
    fig_growth = go.Figure()
    fig_growth.add_trace(go.Scatter(x=all_days, y=word_studied_cum, mode='lines', name='Words Studied (Cumulative)', line=dict(width=3)))
    fig_growth.add_trace(go.Scatter(x=all_days, y=word_learned_cum, mode='lines', name='Words Learned (Cumulative Ever, >R)', line=dict(width=3)))
    fig_growth.update_layout(
        title='Words: Studied vs Learned Over Time (UTC)',
        xaxis_title='Date (UTC)',
        yaxis_title='Unique Words',
        hovermode='x unified',
        template='plotly_white',
    )
    fig_growth.show()

    fig_time = make_subplots(specs=[[{'secondary_y': True}]])
    fig_time.add_trace(go.Bar(x=all_days, y=word_span_daily, name='Daily Session Span Hours'), secondary_y=False)
    fig_time.add_trace(go.Scatter(x=all_days, y=word_span_cum, name='Cumulative Session Span Hours', line=dict(width=3)), secondary_y=True)
    fig_time.update_layout(
        title='Words: Study Time from Session Span (UTC)',
        xaxis_title='Date (UTC)',
        hovermode='x unified',
        template='plotly_white',
    )
    fig_time.update_yaxes(title_text='Daily Hours', secondary_y=False)
    fig_time.update_yaxes(title_text='Cumulative Hours', secondary_y=True)
    fig_time.show()

else:
    fig_conj = go.Figure()
    fig_conj.add_trace(go.Scatter(
        x=all_days,
        y=conjugation_learned_point_in_time,
        mode='lines',
        name='Conjugation Learned (Point-in-Time)',
        line=dict(width=3),
    ))
    fig_conj.update_layout(
        title='Conjugation: Learned Over Time (Point-in-Time, UTC)',
        xaxis_title='Date (UTC)',
        yaxis_title='Unique Verbs Remembered (both tenses > R_TARGET)',
        hovermode='x unified',
        template='plotly_white',
    )
    fig_conj.show()

    fig_time = make_subplots(specs=[[{'secondary_y': True}]])
    fig_time.add_trace(go.Bar(x=all_days, y=conj_span_daily, name='Daily Session Span Hours'), secondary_y=False)
    fig_time.add_trace(go.Scatter(x=all_days, y=conj_span_cum, name='Cumulative Session Span Hours', line=dict(width=3)), secondary_y=True)
    fig_time.update_layout(
        title='Conjugation: Study Time from Session Span (UTC)',
        xaxis_title='Date (UTC)',
        hovermode='x unified',
        template='plotly_white',
    )
    fig_time.update_yaxes(title_text='Daily Hours', secondary_y=False)
    fig_time.update_yaxes(title_text='Cumulative Hours', secondary_y=True)
    fig_time.show()


In [ ]:
summary = pd.DataFrame({
    'day_utc': all_days,
    'words_studied_cum': word_studied_cum.values,
    'words_learned_cum_ever': word_learned_cum.values,
    'conjugation_learned_point_in_time': conjugation_learned_point_in_time.values,
    'word_session_span_hours_daily': word_span_daily.values,
    'verb_session_span_hours_daily': conj_span_daily.values,
})

display(summary.tail(20))
